In [123]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [125]:
poblacion_teorica = pd.read_excel("../data/processed/proyeccion_problemas_de_salud.xlsx")

poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]
poblacion_teorica = poblacion_teorica.query("Estrato == 'Pais'")

DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico").sort_index()
# Selecciona solamente la poblacion prevista
poblacion_teorica = poblacion_teorica[[i for i in range(2017, 2036)]]

In [126]:
egresos_int = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_int = egresos_int[egresos_int["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_int = egresos_int.query("ANO_EGRESO >= 2017")

In [127]:
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Separación de población teórica en hospitalizados y ambulatorios

In [128]:
# De los casos totales, se asume que una parte (0.8) se resuelve en hospitalizacion
RATIO_HOSPITALIZADOS = 0.8
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

poblacion_teorica_hospitalizados = poblacion_teorica * RATIO_HOSPITALIZADOS
poblacion_teorica_ambulatorio = poblacion_teorica * RATIO_AMBULATORIO

In [129]:
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [130]:
tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

## Obtención de brecha de atención del país

En este apartado se quiere saber si el país ha suplido la necesidad de atención para cada problema
de salud. Para esto, se contrastará la cantidad de casos atendidos a nivel país en una hospitalización con
los casos teóricos según la incidencia. La fórmula será:

$ Brecha = \frac{Casos Atendidos Reales - Casos Teoricos}{Casos Teoricos}$

En el caso que la brecha sea positiva, entonces $Casos Atendidos Reales > Casos Teoricos$, y el pais
ha podido suplir la demanda sanitaria del problema de salud. En caso contrario, el país ha sido
incapaz de suplir la demanada para resolver estos problemas de salud. Además, se puede saber
el porcentaje de los casos teóricos que ha suplido el país. Por ejemplo:

$Brecha = 1.12$ Indica que el país ha resolvido todos los casos teóricos del problema de salud.
Además, ha resuelto un 12% más que los casos teóricos.

In [138]:
print(tabla_dinamica_egresos_pais.index == poblacion_teorica_hospitalizados.index)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]


Los indices de ambas tablas son iguales, por lo que se pueden hacer operaciones entre dfs.

In [131]:
brecha_pais = (
    tabla_dinamica_egresos_pais - poblacion_teorica_hospitalizados.round(0)
) / poblacion_teorica_hospitalizados.round(0)

In [132]:
brecha_pais.style.format(precision=2).background_gradient(
    axis=None, vmin=-1, vmax=1, cmap="coolwarm"
)

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
DIAG1,,,,,,,,,,,,,,,,,,,
C33X,0.67,0.20,0.13,-0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C340,-0.84,-0.86,-0.86,-0.90,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C341,-0.63,-0.54,-0.53,-0.71,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C342,-0.95,-0.93,-0.95,-0.96,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C343,-0.75,-0.72,-0.74,-0.79,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C381,-0.77,-0.77,-0.72,-0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C384,6.80,7.80,9.40,7.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C450,15.20,14.20,14.00,9.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C780,-0.63,-0.63,-0.59,-0.70,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Los resultados indican que en gran parte de los diagnósticos faltó suplir la demanada del problema
de salud (ej: C340, C341, C342, C343, etc). Los resultados se pueden dividir en distintos tipos
de
problema de salud:

1. **Cáncer de Pulmón (C33X a C782)**: En todos los diagnósticos faltó suplir la necesidad del
país (exceptuando C384, C450 y E848). EL diagnóstico con una mayor necesidad de atención fue
el C342, con una brecha máxima de -0.96 (faltó atender el 96% de la necesidad teórica).

2. **Patologías Valvulares (I051 a I712)**: En todos los diagnósticos faltó suplir la necesidad del
país (exceptuando I420). El diagnóstico con una mayor necesidad de atención fue I051, con una
brecha máxima de -0.98 (faltó atender el 98% de la necesidad teórica)

In [145]:
display(tabla_dinamica_egresos_pais[tabla_dinamica_egresos_pais.index == "I051"])
display(poblacion_teorica_hospitalizados[poblacion_teorica_hospitalizados.index == "I051"])

ANO_EGRESO,2017,2018,2019,2020
DIAG1,,,,
I051,22,39,26,26


,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
Diagnostico,,,,,,,,,,,,,,,,,,,
I051,1124.30748,1144.585761,1166.304465,1187.735242,1201.167278,1210.335486,1218.412665,1226.072452,1233.432411,1240.504139,1247.278603,1253.741213,1259.879639,1265.682041,1271.243273,1276.485449,1281.402404,1285.990476,1290.24942


Al observar el diagnóstico I051, se puede apreciar que en el año 2020 se tenía una población teórica
de 1188. Sin embargo, solamente se atendieron 26 pacientes en ese año.

In [170]:
teorica_hospitalizados_ppt = (
    poblacion_teorica_hospitalizados.round(0).fillna(0).astype(int).astype(str)
)
tabla_dinamica_egresos_pais_ppt = tabla_dinamica_egresos_pais.copy().astype(str)
tabla_dinamica_egresos_pais_ppt[[i for i in range(2021, 2036)]] = "0"

brecha_pais_ppt = brecha_pais.copy()
brecha_pais_ppt = brecha_pais_ppt.round(2).astype(str).replace("nan", "-")

In [179]:
tabla_dinamica_egresos_pais_ppt + "; " + teorica_hospitalizados_ppt + "; (" + brecha_pais_ppt + ")"

ANO_EGRESO,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
DIAG1,,,,,,,,,,,,,,,,,,,
C33X,25; 15; (0.67),18; 15; (0.2),17; 15; (0.13),14; 16; (-0.12),0; 16; (-),0; 16; (-),0; 16; (-),0; 16; (-),0; 16; (-),0; 16; (-),0; 16; (-),0; 16; (-),0; 17; (-),0; 17; (-),0; 17; (-),0; 17; (-),0; 17; (-),0; 17; (-),0; 17; (-)
C340,163; 1017; (-0.84),143; 1035; (-0.86),148; 1055; (-0.86),103; 1074; (-0.9),0; 1086; (-),0; 1095; (-),0; 1102; (-),0; 1109; (-),0; 1115; (-),0; 1122; (-),0; 1128; (-),0; 1134; (-),0; 1139; (-),0; 1145; (-),0; 1150; (-),0; 1154; (-),0; 1159; (-),0; 1163; (-),0; 1167; (-)
C341,372; 1017; (-0.63),478; 1035; (-0.54),493; 1055; (-0.53),316; 1074; (-0.71),0; 1086; (-),0; 1095; (-),0; 1102; (-),0; 1109; (-),0; 1115; (-),0; 1122; (-),0; 1128; (-),0; 1134; (-),0; 1139; (-),0; 1145; (-),0; 1150; (-),0; 1154; (-),0; 1159; (-),0; 1163; (-),0; 1167; (-)
C342,50; 1017; (-0.95),68; 1035; (-0.93),52; 1055; (-0.95),48; 1074; (-0.96),0; 1086; (-),0; 1095; (-),0; 1102; (-),0; 1109; (-),0; 1115; (-),0; 1122; (-),0; 1128; (-),0; 1134; (-),0; 1139; (-),0; 1145; (-),0; 1150; (-),0; 1154; (-),0; 1159; (-),0; 1163; (-),0; 1167; (-)
C343,258; 1017; (-0.75),293; 1035; (-0.72),279; 1055; (-0.74),223; 1074; (-0.79),0; 1086; (-),0; 1095; (-),0; 1102; (-),0; 1109; (-),0; 1115; (-),0; 1122; (-),0; 1128; (-),0; 1134; (-),0; 1139; (-),0; 1145; (-),0; 1150; (-),0; 1154; (-),0; 1159; (-),0; 1163; (-),0; 1167; (-)
C381,41; 177; (-0.77),41; 180; (-0.77),52; 183; (-0.72),42; 187; (-0.78),0; 189; (-),0; 190; (-),0; 192; (-),0; 193; (-),0; 194; (-),0; 195; (-),0; 196; (-),0; 197; (-),0; 198; (-),0; 199; (-),0; 200; (-),0; 201; (-),0; 202; (-),0; 202; (-),0; 203; (-)
C384,39; 5; (6.8),44; 5; (7.8),52; 5; (9.4),43; 5; (7.6),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-)
C450,81; 5; (15.2),76; 5; (14.2),75; 5; (14.0),53; 5; (9.6),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-),0; 6; (-)
C780,373; 1017; (-0.63),379; 1035; (-0.63),430; 1055; (-0.59),326; 1074; (-0.7),0; 1086; (-),0; 1095; (-),0; 1102; (-),0; 1109; (-),0; 1115; (-),0; 1122; (-),0; 1128; (-),0; 1134; (-),0; 1139; (-),0; 1145; (-),0; 1150; (-),0; 1154; (-),0; 1159; (-),0; 1163; (-),0; 1167; (-)
